# 데이터 전처리(3) - 데이터 재구조화

- 피벗 테이블
- melt 
- stack, unstack
- 교차표 (범주형 데이터 빈도)

In [15]:
from pandas import DataFrame, read_excel, merge, concat, pivot_table, crosstab, melt

## 피벗테이블

### 1. 샘플 데이터 가져오기

In [16]:
df = read_excel("city.xlsx")
df

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,2632035,수도권


### 2. 피벗테이블 기본

인덱스, 컬럼, 값으로 사용할 필드를 각각 지정하여 데이터를 재배치

In [17]:
pivot_table(df,# 피벗할 데이터프레임
            index='도시', # 행 위치에 들어갈 열 -> 도시가 인덱스가 된다. 
            columns='연도', # 열 위치에 들어갈 열 
            values='인구' # 데이터로 사용할 열 -> 값이다. 

)

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,2632035.0,2890451.0


3. 중복데이터의 집계 방법 지정하기

In [18]:
pivot_table(df,
            index='지역', 
            columns='연도', 
            values='인구',
            aggfunc='mean' # 통계학적 의미의 평균
)

연도,2005,2010,2015
지역,,,
경상권,3512547.0,3393191.0,3448737.0
수도권,9762546.0,6131758.5,6397381.5


4. 복수집계함수 지정

In [19]:
pivot_table(df,
            index='지역', 
            columns='연도', 
            values='인구',
            aggfunc=['mean','sum'] # 여러개일때는 리스트 타입으로 묶는다
)

mean                            sum                    
연도        2005       2010       2015     2005      2010      2015
지역                                                               
경상권  3512547.0  3393191.0  3448737.0  3512547   3393191   3448737
수도권  9762546.0  6131758.5  6397381.5  9762546  12263517  12794763

In [20]:
pivot_table(df,
            index=['지역','연도'], 
            columns='도시', 
            values='인구',
            aggfunc=['mean','sum'] # 통계학적 의미의 평균
)

mean                              sum                      
도시               부산         서울         인천         부산         서울         인천
지역  연도                                                                    
경상권 2005  3512547.0        NaN        NaN  3512547.0        NaN        NaN
    2010  3393191.0        NaN        NaN  3393191.0        NaN        NaN
    2015  3448737.0        NaN        NaN  3448737.0        NaN        NaN
수도권 2005        NaN  9762546.0        NaN        NaN  9762546.0        NaN
    2010        NaN  9631482.0  2632035.0        NaN  9631482.0  2632035.0
    2015        NaN  9904312.0  2890451.0        NaN  9904312.0  2890451.0

## melt 

데이터 테이블의 컬럼 이름을 변수화한 형태로, 피벗테이블을 분리한 형태다.

In [21]:
# 샘플 피벗 테이블

pivot_df = pivot_table(df,
                       index = '연도',
                       columns='지역',
                       values='인구',
                       aggfunc = 'mean')
pivot_df

지역,경상권,수도권
연도,,
2005,3512547.0,9762546.0
2010,3393191.0,6131758.5
2015,3448737.0,6397381.5


In [26]:
# 데이터프레임의 인덱스를 일반 컬럼으로 설정
# reset_index()는 인덱스를 리셋해준다. 인덱스를 사용할 수 없어서 저렇게 해야함. 
pivot_df2 = pivot_df.reset_index()
pivot_df2

지역,연도,경상권,수도권
0,2005,3512547.0,9762546.0
1,2010,3393191.0,6131758.5
2,2015,3448737.0,6397381.5


피벗테이블 분리
- id_vars : 인덱스로 사요할 컬럼명, 반드시 컬럼만 가능하다(인덱스 불가)
- valuse_vars : 분리할 컬럼 이름들

In [27]:
# 피벗테이블 분리
mdf = melt(pivot_df2, id_vars=['연도'],value_vars=['경상권','수도권'])
mdf

,연도,지역,value
0,2005,경상권,3512547.0
1,2010,경상권,3393191.0
2,2015,경상권,3448737.0
3,2005,수도권,9762546.0
4,2010,수도권,6131758.5
5,2015,수도권,6397381.5


In [28]:
# 피벗 테이블 분리 및 필드 이름 지정
mdf = melt(pivot_df2, id_vars=['연도'],value_vars=['경상권','수도권'],
           var_name='구분',value_name='인구수')
mdf

,연도,구분,인구수
0,2005,경상권,3512547.0
1,2010,경상권,3393191.0
2,2015,경상권,3448737.0
3,2005,수도권,9762546.0
4,2010,수도권,6131758.5
5,2015,수도권,6397381.5


## stack, unstsack



데이터 분리 (stack)
모든 변수를 하나의 변수로 쌓아놓는 처리 
컬럼 이름과 값을 하나의 필드로 묶어서 시리즈 객체로 반환한다. 

그러면 인덱스는 모야?


In [29]:
df = read_excel("https://data.hossam.kr/C02/body.xlsx", index_col="name")
df

,sex,height,weight
name,,,
Lee,M,175,98.0
Park,F,167,48.0
Hong,M,180,NaN
Kim,F,162,55.0
Nam,M,172,85.0


stack 처리
- 리턴 결과를 Series 객체도 리턴한다.

In [30]:
st = df.stack()
st

name        
Lee   sex          M
      height     175
      weight    98.0
Park  sex          F
      height     167
      weight    48.0
Hong  sex          M
      height     180
Kim   sex          F
      height     162
      weight    55.0
Nam   sex          M
      height     172
      weight    85.0
dtype: object

In [32]:
# stack 결과글 dataFrame으로 만들기
df2 = DataFrame(st)
df2

0
name             
Lee  sex        M
     height   175
     weight  98.0
Park sex        F
     height   167
     weight  48.0
Hong sex        M
     height   180
Kim  sex        F
     height   162
     weight  55.0
Nam  sex        M
     height   172
     weight  85.0

In [33]:
# 빈 값(NaN) 처리
# dropna=False 파라미터를 적용하면 빈값을 유지한다. 기본값은 True (빈값 삭제)

st2 = df.stack(dropna=False)
st2

name        
Lee   sex          M
      height     175
      weight    98.0
Park  sex          F
      height     167
      weight    48.0
Hong  sex          M
      height     180
      weight     NaN
Kim   sex          F
      height     162
      weight    55.0
Nam   sex          M
      height     172
      weight    85.0
dtype: object

In [34]:
# stack 결과를 원래대로 되돌림
st.unstack

<bound method Series.unstack of name        
Lee   sex          M
      height     175
      weight    98.0
Park  sex          F
      height     167
      weight    48.0
Hong  sex          M
      height     180
Kim   sex          F
      height     162
      weight    55.0
Nam   sex          M
      height     172
      weight    85.0
dtype: object>

In [35]:
# unstack을 수행하면서 빈 값을 다른 값으로 대체
st.unstack(fill_value=100)

,sex,height,weight
name,,,
Lee,M,175,98.0
Park,F,167,48.0
Hong,M,180,100
Kim,F,162,55.0
Nam,M,172,85.0


## 교차표(crosstab)

범주형 자료를 갖는 데이터에 대해 각 범주별로 빈도수를 계산하여 표현한 표

In [22]:
df = read_excel("MW.xlsx")
df

,gender,score
0,M,A
1,M,C
2,M,B
3,M,B
4,W,A
5,W,C
6,W,C
7,W,B


2. 교차표 만들기

index 파라미터와 columns 파라미터를 지정한다.

In [23]:
crosstab(index=df['gender'],columns=df['score'])
# 인덱스는 gender로 colums는 ABC학점으로 각각 몇명인지 나오게 한다. 

score,A,B,C
gender,,,
M,1,2,1
W,1,1,2


3. 파라미터 설정
- rownames : 인덱스의 이름 설정
- colnames : 컬럼의 이름 설정
- margins : 집계 결과 포함 여부 (True/False)

In [24]:
crosstab(index=df['gender'],columns=df['score'],
         rownames=['성별'],colnames=['학점'],margins=True)

학점,A,B,C,All
성별,,,,
M,1,2,1,4
W,1,1,2,4
All,2,3,3,8


4. 비율표시
- normalize = True 파라미터 사용

In [25]:
crosstab(index=df['gender'], columns=df['score'],
         rownames=['성별'], colnames=['점수'], normalize=True)

점수,A,B,C
성별,,,
M,0.125,0.250,0.125
W,0.125,0.125,0.250
